# Sprint 2 - Webinar 3: Formulación de Preguntas de Negocio y Preparación de Datos en Excel/Sheets (80 min)

**Fecha de creación:** 2025-09-14 01:18  
**Dataset base:** `almacen_ventas.xlsx`

---
## Objetivos de Aprendizaje
- Traducir escenarios de negocio en **preguntas analíticas claras** y **métricas** medibles.
- Diferenciar métricas **absolutas** vs **relativas** e identificar **KPI/OKR** y *guardrail metrics*.
- Distinguir **leading** vs **lagging indicators** y redactar preguntas enfocadas.
- **Preparar** y **combinar** datasets (XLOOKUP/VLOOKUP), aplicar limpieza categórica (TRIM/PROPER/CLEAN).
- Resumir datos con **SUMIF/SUMIFS/COUNTIF/AVERAGEIF** y organizar un libro “analysis‑ready”.
- Apoyarse de un **LLM** para refinar stakeholders, métricas y fórmulas (usándolo con criterio).

> **Tiempo total:** 80 minutos — pensado para trabajar sobre `almacen_ventas.xlsx`.


---
## Agenda (80 min)
1) Introducción al contexto analítico (10 min)  
2) Preguntas de negocio y métricas (25 min)  
3) Preparación y exploración del dataset (25 min)  
4) Resumen y ejercicios de cierre (20 min)


---
## 0. Setup (Docente)
Antes de iniciar, asegúrate de tener el archivo **`almacen_ventas.xlsx`** en tu equipo.  
Si estás usando este notebook en local (con Python), la siguiente celda demuestra cómo leerlo con `pandas`.
Si trabajas en **Excel/Google Sheets**, simplemente abre el archivo y sigue las instrucciones en las hojas.


In [ ]:
# (Opcional para docente) Cargar datos con pandas si deseas mostrar ejemplos en Python
import pandas as pd
from pathlib import Path

xlsx_path = Path('/mnt/data/almacen_ventas.xlsx')  # ajusta la ruta si tu archivo está en otra ubicación
assert xlsx_path.exists(), "No se encuentra el archivo almacen_ventas.xlsx en la ruta especificada."

raw = pd.read_excel(xlsx_path, sheet_name='Raw Data')
customers = pd.read_excel(xlsx_path, sheet_name='Customers')
readme = pd.read_excel(xlsx_path, sheet_name='README')

raw.head(3), customers.head(3)


---
# 1) Analytical Questions in a Business Context (10 + 25 min)

### 1.1 Framing Data Questions from Business Needs (5 min)
**Idea clave:** convertir necesidades del negocio en preguntas analíticas claras.
- Identifica **stakeholders** (quién decide).
- Explicita la **decisión** a tomar y el **horizonte temporal**.
- Revisa **campos disponibles** del dataset y mapea cómo responden preguntas.

**Ejemplos rápidos (para discutir en clase):**
- Stakeholder: Gerente de Ventas → Decisión: reasignar presupuesto regional trimestral → Dataset: `Date, Region, Total_Sales`.
- Stakeholder: Marketing → Decisión: campaña por producto → Dataset: `Product, Units_Sold, Region`.

**Ejercicio 1 (5 min, en parejas):**
Para cada mini-escenario, escribe:
1) Stakeholder
2) Pregunta clave (1 línea, clara)
3) Campos del dataset que usarías

a) “Necesito saber qué producto impulsar en el próximo mes en la región West.”  
b) “Quiero entender el desempeño mensual por vendedor.”  
c) “¿Qué categoría sostiene el crecimiento del trimestre?”


### 1.2 Matching Business Questions with the Right Metrics (10 min)
**Métricas básicas:** total (SUM), promedio (AVERAGE), crecimiento % (MoM/YoY), razones/ratios, varianza, percentiles.  
**KPI vs OKR:** KPI mide desempeño continuo; OKR formula objetivos + resultados clave.  
**Guardrail metrics:** métricas de seguridad para evitar impactos indeseados (p.ej., churn, NPS).  
**Absolutas vs relativas:** totales vs tasas/porcentajes.

**Ejercicio 2 (10 min, en Excel/Sheets):**
En `Raw Data` crear columna `Month`:
- Excel: `=TEXT([@Date],"mmm-yyyy")`  (o usar **Power Query** si prefieres)
- Sheets: `=TEXT(A2,"mmm-yyyy")`

Responde con fórmulas (no Pivot, por ahora):
1) **Total de ventas por producto** (usa `SUMIF` o `SUMIFS`)  
2) **Promedio de ventas por región** (`AVERAGEIF`)  
3) **Crecimiento % Mes a Mes** de *Total_Sales*:
   - Construye una tabla de Ventas por `Month`
   - Fórmula de crecimiento: `(MesActual - MesAnterior) / MesAnterior`

> Deja las fórmulas visibles y nombra los rangos para facilitar su lectura.


### 1.3 Leading vs. Lagging Indicators (5 min)
- **Leading** (anticipan): visitas al sitio, leads, unidades vendidas, cotizaciones.  
- **Lagging** (confirman): ingresos, margen, utilidad, ventas cerradas.

**Ejercicio 3 (3–5 min):**
En una tabla de dos columnas, clasifica como *Leading* o *Lagging* los campos de `Raw Data`:
- `Units_Sold` →
- `Total_Sales` →
- `Date` (cuando mide período consolidado) →
- `Region` (no es métrica, es dimensión, ¿cómo la usarías para indicadores?) →

Justifica brevemente en 1 oración por campo.


### 1.4 Writing Focused Analysis Questions (3 min)
Estructura: **[Métrica] para [grupo/segmento] en [tiempo]**.  
- Vago: “¿Cómo vamos en ventas?”  
- Enfocado: “**Total_Sales** por **Product** en **West** durante **jun-2024** (comparado con may-2024).”

**Ejercicio 4 (5 min):** Reescribe 3–5 preguntas vagas a formato listo para Pivot/función:
1) “Quiero saber qué tan bien vende Sofía.”  
2) “Quiero entender a los clientes.”  
3) “¿Qué región es la mejor?”


### 1.5 🤖 AI: Using an LLM for stakeholder & metrics (2 min)
Prompt sugerido (copiar/pegar en tu LLM favorito):
> *Tengo un dataset con columnas Date, Region, Product, Units_Sold, Unit_Price, Salesperson, Total_Sales. Soy el Gerente de Ventas. Necesito priorizar presupuesto por región para el próximo trimestre. Sugiere: (1) objetivo y pregunta clave, (2) métricas y cálculos (incluye crecimiento y tasas), (3) qué tablas/resúmenes construirías para decidir.*

**Discusión:** ¿Qué aceptarías del LLM? ¿Qué validarías manualmente?


---
# 2) Preparing and Exploring the Dataset (25 min)

### 2.1 Importing and Exploring a New Dataset (5 min)
Abrir hoja **Customers** y revisar:
- `Customer_ID`, `Region`, `Customer_Type` (Retail/Wholesale)
- ¿Cómo podría relacionarse con `Raw Data`?

**Ejercicio 5 (5 min):** Lista 3–4 posibles conexiones entre hojas:
- Por `Region`
- Por (futuro) `Customer_ID` en ventas
- Por `Month` a nivel de segmentación comercial
- Por `Customer_Type` para analizar mix Retail vs Wholesale


### 2.2 Combining Two Related Datasets (8 min)
**Lookup exacto** vs aproximado. Lookup tables (dimensiones) vs tablas transaccionales.

**Paso a paso con `XLOOKUP` (o `VLOOKUP`):**
1. En `Raw Data`, inserta columna `Customer_Type` (o `Region_Type` si prefieres).  
2. Fórmula (ejemplo por `Region`):  
   - Excel: `=XLOOKUP([@Region],Customers!B:B,Customers!C:C,"NoMatch",0)`  
   - Sheets (XLOOKUP): `=XLOOKUP(C2,Customers!B:B,Customers!C:C,"NoMatch",0)`  
   - o VLOOKUP (asumiendo tabla ordenada): `=VLOOKUP(C2,Customers!B:C,2,FALSE)`
3. **Verificación y troubleshooting:**  
   - Cuenta matches: crea un conteo de `#N/A` con `=COUNTIF(Rango,"#N/A")`  
   - Duplica: usa `Quitar duplicados` o conditional formatting

**Ejercicio 6 (8–10 min):**
- Haz el *perfect match* primero (sin errores).  
- Luego introduce un error tipográfico (“Nort” por “North”) y observa `#N/A`.  
- Documenta en una hoja “JOIN_DOC”:
  - supuestos del join (clave, cardinalidad, exactitud)
  - limitaciones detectadas (errores, duplicados)


### 2.3 Organizing the Spreadsheet for Analysis (5 min)
Buenas prácticas:
- Renombrar y color‑code tabs (Datos, Dimensiones, Resúmenes, README).
- Congelar encabezados, ajustar anchos, formato de miles y porcentajes.
- **Declarative names** en columnas: `Total_Sales`, `Units_Sold` (evitar “col1”).
- Validación de datos para `Region` ∈ {North, South, East, West}.
- **Helper columns**: `Month`, `Quarter`, `Product_Category` (si aplica).

**Ejercicio 7 (5 min):**
- Renombra: `Sales_Raw`, `Customers`, `README`, `Cleaned_Data`, `Summary`.
- Crea validaciones (listas) para `Region` y `Product`.
- En `README`, documenta la estructura, pestañas y convenciones.


### 2.4 Preparing Categorical Data for Grouping (5 min)
**Limpieza en hoja `Cleaned_Data`** referenciando `Sales_Raw` (no copiar/pegar manual):
- `=TRIM(Sales_Raw!B2)` elimina espacios extra
- `=PROPER(Sales_Raw!C2)` capitaliza
- `=CLEAN(...)` remueve caracteres no imprimibles
- Plantilla de chequeo: `=IF(ISBLANK(A2),"Missing","OK")`

**Ejercicio 8 (5 min):**
- Implementa una plantilla de limpieza con columnas: `Original`, `Trimmed`, `Proper`, `Check`  
- Añade una columna **`Issue_Flag`** con reglas: si `Region` no está en la lista válida → “Invalid Region”.
- Guarda esta hoja como **plantilla reutilizable**.


### 2.5 🤖 AI para fórmulas & errores (2 min)
Ejemplo de prompt (para depurar fórmulas):
> *Tengo esta fórmula que devuelve #N/A/ERROR en Excel/Sheets: =XLOOKUP(C2,Customers!B:B,Customers!C:C). Mi objetivo es traer Customer_Type por Region. ¿Qué revisaría y cómo la reescribirías? Dame 2 alternativas y cómo validar el resultado.*


---
# 3) Summarizing Data with Functions and Filters (ráfagas guiadas)

### 3.1 Filtering and Exploring Patterns
- Usa filtros combinados (por `Region` y `Month`) para aislar subconjuntos.
- Identifica patrones visuales rápidos (colores/heatmap, barras in-cell).

**Ejercicio 9 (5 min):**
Filtra un mes específico y responde:  
“¿Qué **Product** tuvo mayor `Total_Sales` en **South**?”


### 3.2 Conditional Aggregation Functions
- `SUMIF/COUNTIF/AVERAGEIF` (1 condición) vs `SUMIFS/COUNTIFS/AVERAGEIFS` (multi-condición).
- Usa referencias absolutas/relativas con intención (F4 en Excel).

**Ejercicio 10 (8 min):**
- Total de ventas por **Region** con `SUMIF`  
- Promedio de `Units_Sold` por **Product** con `AVERAGEIF`  
- Resalta duplicados y outliers con **formato condicional** (reglas > umbral)


### 3.3 Comparing Segments with Multiple Conditions
Construye una tabla de resumen usando `SUMIFS` por `Month` y dos regiones (ej. West vs East).

**Ejercicio 11 (7–8 min):**
Tabla: filas = `Month`; columnas = `West`, `East`; valores = `SUMIFS(Total_Sales, Region, "West", Month, [Mes])` …  
**Pregunta:** ¿Cuál región crece más en el último trimestre?


### 3.4 Creating Summary Sheets for Key Metrics
- Hoja `Summary`: KPIs (3), tabla breve y 1 mini‑gráfico.
- Ajusta para la audiencia (equipo vs C‑level): pocos números, limpia y clara.

**Ejercicio 12 (7–8 min):**
- KPIs sugeridos: **Ventas totales**, **Ticket promedio** (Total_Sales/Units_Sold), **% crecimiento último mes**.
- 1 gráfico: columnas apiladas por `Month` y `Region` o por `Product` top‑N.
- Escribe 2 bullets de insight (contexto → hallazgo → implicación).


---
## 4) Mini‑Caso de Cierre (15–20 min)
**Stakeholder:** CEO  
**Pregunta:** ¿Cómo evolucionaron las ventas de *Laptop* y *Smartphone* en el último trimestre por región?

**Tareas:**
1. Asegura columna `Month` correcta.
2. Calcula `Total_Sales` por `Month`, `Region`, `Product` (filtro en Laptop/Smartphone).
3. Crea gráfico comparativo.
4. Redacta **2–3 insights** útiles para decisión de presupuesto.

> **Entrega sugerida:** hoja `Summary` con KPIs, tabla, gráfico y sección “Insights”.


---
## Apéndice: Tips de clase / Soluciones guía (para el Tutor)
- Verifica que el `XLOOKUP` tenga coincidencia exacta (0/FALSE) y que no haya espacios ocultos → usa `TRIM()` en origen y destino.
- Para crecimiento **MoM**: ordena los meses cronológicamente (usa una columna `Month_Key = YEAR*100+MONTH`).
- Usa **nombre de rangos** o **Tablas** en Excel para fórmulas legibles (`TablaVentas[Total_Sales]`).
- Si el dataset tiene pocos meses, explica diferencia entre **tasa de crecimiento** y **variación absoluta**.
- **Leading vs Lagging**: valida con ejemplos del propio negocio de los estudiantes para afianzar.
